In [48]:
!pip install tfx

# Data Pipeline components for Production ML
- First three steps of a production machine learning project - Data Ingestion , Data Validation , Data Transformation.
#### Build the production data pipeline by :
- performing feature selection
- Ingesting the dataset
- Generate the statistics of the dataset
- Creating a schema as per the domain knowledge
- visualize the dataset anomalies
- preprocessing , transforming and Engineering the features
- Tracking the data pipeline using ML metadata



## Imports

In [49]:
import tensorflow as tf
from tfx import v1 as tfx

# Tfx libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For feature selection
from sklearn.feature_selection import SelectKBest , f_classif

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from tensorflow_metadata.proto.v0 import schema_pb2
import os
import pprint
import tempfile
import pandas as pd

# versions of the tensorflow module's
print(f"tensorflow version : {tf.__version__}")
print(f"tensorflow extended version : {tfx.__version__}")
print(f"tensorflow validation : {tfdv.__version__}")
print(f"Tensorflow transform version : {tft.__version__}")

tensorflow version : 2.13.1
tensorflow extended version : 1.14.0
tensorflow validation : 1.14.0
Tensorflow transform version : 1.14.0


## Download the dataset

In [50]:
# download the dataset
!wget -nc https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv -P {TRAINING_DIR}

File ‘{TRAINING_DIR}/dataset.csv’ already there; not retrieving.



## Feature Selection
- Reduce the number of features so that the model can be trained easily with increased accuracy.

In [51]:
# Load the dataset
df = pd.read_csv("/content/{TRAINING_DIR}/dataset.csv")
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4


In [52]:
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

- Most of the features are numeric in nature.
- so remove the object features from the dataset.
- Create a copy of the dataset to make the changes in the original dataset.

In [53]:
df_dumpy = df.copy()

# Categorical columns
cate_column = ['Wilderness_Area','Soil_Type']
# Label column
label_column = ['Cover_Type']

# Drop the categorical and label features from the dumpy dataset copy
df_dumpy.drop(cate_column,axis = 1,inplace = True)
df_dumpy.drop(label_column,axis = 1 , inplace = True)

df_dumpy.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


In [54]:
# Set the target and feature values from the dumpy dataset
y = df[label_column].values
x = df_dumpy.values

## Feature Selection
- Select the features from the dataset using SelectKBest score each input feature against the target variable.

In [55]:
from sklearn.feature_selection import SelectKBest, chi2

select_k_best = SelectKBest(k=8)
x_new = select_k_best.fit_transform(x,y)
feature_mask = select_k_best.get_support()

required_cols = pd.DataFrame({'columns' : df_dumpy.columns, 'Retained_Features': feature_mask})
required_cols

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,columns,Retained_Features
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


- Now select this subset of features from the original dataframe and save it to the new directory.

In [56]:
# Set the paths to the reduced dataset
TRAINING_DIR_FSELECT = f"{{TRAINING_DIR}}/fselect"
TRAINING_DATA_FSELECT = f"{TRAINING_DIR_FSELECT}/dataset.csv"

# Create this directory
if not os.path.exists(TRAINING_DIR_FSELECT):
  os.mkdir(TRAINING_DIR_FSELECT)

In [57]:
# Get the feature names from the selectkbest
feature_names = list(df_dumpy.columns[feature_mask])

# Append the categorical and label columns
feature_names = feature_names + cate_column + label_column

# Select the selected subset of columns
df_select = df[feature_names]

# Write the csv with the selected features , categorical and label features as the new dataset
df_select.to_csv(TRAINING_DATA_FSELECT,index = False)

df_select.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,3,258,0,510,221,232,6279,Rawah,C7745,4
1,2590,2,212,-6,390,220,235,6225,Rawah,C7745,4
2,2804,9,268,65,3180,234,238,6121,Rawah,C4744,1
3,2785,18,242,118,3090,238,238,6211,Rawah,C7746,1
4,2595,2,153,-1,391,220,234,6172,Rawah,C7745,4


## Data Pipeline
- with the selected subset of features,now start building the data pipeline.
- This involves ingesting,validating and transforming of the data.
- Using the TFX components build the pipeline.

In [58]:
# Location of the pipeline
pipeline_dir = "./pipeline"
if not os.path.exists(pipeline_dir):
  _pipeline_dir = os.mkdir(pipeline_dir)

# Use the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root = _pipeline_dir)

### Generating examples
- First step in the pipeline is to ingest the data.
- using ExampleGen convert the raw into TFRecords for faster computation in the later stages of the pipeline.

In [59]:
example_gen = tfx.components.CsvExampleGen(
    input_base = TRAINING_DIR_FSELECT
)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Computing Statistics
- will give the statistics of the data
- allows to observe and analyze characteristics of the data and visualization of the data

In [60]:
# Statistics Gen
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- Having zeros in the covtype column is as expect because the range is between 0-6.

In [61]:
# Display the results
context.show(statistics_gen.outputs['statistics'])

### Inferring the Schema
- create a schema to validate the incoming datasets during the training and serving.

In [62]:
# SchemaGen
schema_gen = tfx.components.SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [63]:
# visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Curating the schema
- The inferred schema is able to capture the data types correctly and also to show the expected values for the qualitative data.
- Still we can fine-tune this. We have features where we expect a certain range:
  - Hillshade_9am : 0 to 255
  - Hillshade_Noon : 0 to 255
  - Slope : 0 to 90
  - Cov_Type : 0 to 6
- We have to update the schema to take note of these so the pipeline can detect if invalid values are being fed into model.

In [64]:
try :
  # Get the schema uri
  schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
except IndexError:
  print("context.run() was no-op")
  schema_path = "./pipeline/SchemaGen/schema"
  dir_id = os.listdir(schema_path)[0]
  schema_uri = f'{schema_path}/{dir_id}'

In [65]:
# Get the pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri,"schema.pbtxt"))

In [66]:

# Set the two `Hillshade` features to have a range of 0 to 255
tfdv.set_domain(schema,"Hillshade_9am", schema_pb2.IntDomain(name='Hillshade_9am', min= 0, max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(name='Hillshade_Noon', min= 0, max=255))

# Set the `Slope` feature to have a range of 0 to 90
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(name='Slope', min= 0, max=90))

# Set `Cover_Type` to categorical having minimum value of 0 and maximum value of 6
tfdv.set_domain(schema, "Cover_Type", schema_pb2.IntDomain(name='Cover_Type', min= 0, max= 6, is_categorical= True))
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Schema Environment
- Splitting the environments into two
  - TRAINING
    - Which has both input and ouput,with these we train our model.
  - SERVING
    - It has only input , output has to be predicted by the model.

In [67]:
# Paths for serving data
DATA_DIR = '/content'
SERVING_DIR = f'{DATA_DIR}/serving'
SERVING_DATA_DIR = f'{SERVING_DIR}/serving_dataset.csv'
# Create the directory for the serving environment
if not os.path.exists(SERVING_DIR):
  os.mkdir(SERVING_DIR)

In [68]:
# Read the training dataset
TRAINING_DATA = '/content/{TRAINING_DIR}/dataset.csv'
serving_data = pd.read_csv(TRAINING_DATA,nrows = 100)

# Drop the target label
serving_data.drop(columns='Cover_Type',inplace = True)

# Save the modified dataset
serving_data.to_csv(SERVING_DATA_DIR,index=False)

In [69]:
# Detect the stats options to the curated schema
stats_options = tfdv.StatsOptions(schema = schema, infer_type_from_schema=True)

# Compute the serving dataset to obtain the stats
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA_DIR,stats_options = stats_options)

# Detect the anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats,schema = schema)

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


- as expected the missing column is detected and displayed.Now we have to configure the schema to detect when it is used for training and serving .
- So we have to create environments.

In [70]:
schema.default_environment.append("TRAINING")

schema.default_environment.append("SERVING")

tfdv.get_feature(schema,"Cover_Type").not_in_environment.append("SERVING")



In [71]:
# Now detect the anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats,schema = schema,environment = 'SERVING')
tfdv.display_anomalies(anomalies)

- Now save the curated schema in local Directory so that we can import it to our tfx pipeline.


In [72]:
updated_schema_root = './pipeline/updated_schema'
if not os.path.exists(updated_schema_root):
  os.mkdir(updated_schema_root)

# Create a schema file in the updated schema directory
schema_file = os.path.join(updated_schema_root,'schema.pbtxt')

# Write the curated schema to the created .pbtxt file
tfdv.write_schema_text(schema,schema_file)

- Check the created file has the changes in the file what we have made.

In [73]:
new_schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema = new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [74]:
# Print the list of environment
new_schema.default_environment

['TRAINING', 'SERVING']

### Generate new statistics using the updated schema
- now compute the statistics using the curated schema.
- TFX component interact with each other by getting the artifact information from the metadata store.

In [75]:
# Import the Curated schema file to the ML Metadata
user_schema_importer = tfx.components.ImportSchemaGen(
    schema_file = schema_file
)

context.run(user_schema_importer)
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


- now artifact has been created successfully now you can compute the statistics with curated schema.

In [76]:
statistics_gen_updates = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples'],
    schema = user_schema_importer.outputs['schema']
)

context.run(statistics_gen_updates)
context.show(statistics_gen_updates.outputs['statistics'])

- we can see that the feature "Cover_Type" is shown under the categorical features.

### Check Anomalies
- check if the dataset has any anomalies with respect to the schema.
#### Examplevalidator
- check any anomalies using ExampleValidator.

In [77]:
example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen_updates.outputs['statistics'],
    schema = user_schema_importer.outputs['schema']
)

context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

### Feature Engineering
- Now proceed with to transform the features to a form of suitable features for training a model.
- Include serveral techniques such as scaling and converting strings to vocabulary indices.
- TFX ensures this by generating a graph that will process incoming data both training and infernece.

In [78]:
# Set the constant module filename
_cover_constants_module_file = "cover_constants.py"

In [79]:
%%writefile {_cover_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = [
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
    ]

SCALE_01_FEATURE_KEYS = [
        "Hillshade_9am",
        "Hillshade_Noon",
        "Horizontal_Distance_To_Fire_Points",
    ]

SCALE_Z_FEATURE_KEY = [
        "Elevation",
        "Slope",
        "Horizontal_Distance_To_Roadways",
    ]

VOCAB_FEATURE_KEYS = ["Wilderness_Area"]

HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

LABEL_KEY = "Cover_Type"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting cover_constants.py


### Preprocessing Function
- Transform the features:
  - tft.scale_by_min_max
  - tft.scale_to_0_1
  - tft.scale_to_z_score
  - tft.compute_and_apply_vocabulary
  - tft.hash_strings

In [80]:
# Set the transform module filename

_cover_transform_module_file = "cover_transform.py"


In [81]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEY
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    ### START CODE HERE ###
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling of min_max function
        # Hint: Use tft.scale_by_min_max by passing in the respective column
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling of 0 to 1 function
        # Hint: tft.scale_to_0_1
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling to z score
        # Hint: tft.scale_to_z_score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using vocabulary available in column
        # Hint: Use tft.compute_and_apply_vocabulary
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform by hashing strings into buckets
        # Hint: Use tft.hash_strings with the param hash_buckets set to 10
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)

    ### END CODE HERE ###

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

Overwriting cover_transform.py


## Transform
- Use transform component to perform the transformatioon and generate the transformation graph.

In [47]:
transform = tfx.components.Transform(
    examples = example_gen.outputs['examples'],
    schema = user_schema_importer.outputs['schema'],
    module_file = os.path.abspath("/content/cover_transform.py")
)

context.run(transform,enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)